In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pathlib import Path
import statsmodels.formula.api as smf
sys.path.insert(0, "/home/srs-9/Projects/ms_mri/analysis/thalamus/helpers")

import utils

In [2]:
path = "/home/srs-9/Projects/ms_mri/analysis/thalamus/data0/chaco1_roi_means.csv"
df_chaco = pd.read_csv(path, index_col="subid")
df = pd.read_csv("/home/srs-9/Projects/ms_mri/analysis/thalamus/results/data_wchaco.csv", index_col="subid")

hips_thomas_ref = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/hipsthomas_struct_index.csv", index_col="index"
)["struct"]
thalamic_nuclei = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [15]:
image_root = Path("/home/srs-9/Projects/ms_mri/analysis/thalamus/chaco/images")
for col in df_chaco.columns:
    plt.hist(df_chaco[col])
    plt.xlim([0,0.8])
    plt.title(col)
    plt.savefig(image_root / f"{col}_raw.png")
    plt.close()

In [16]:
transforms = utils.transforms
transforms['3root'] = lambda x: x**(1/3)
trans = "3root"
for col in df_chaco.columns:
    plt.hist(utils.transforms[trans](df_chaco[col]))
    # plt.xlim([0,0.8])
    plt.title(f"{col}, {trans}")
    plt.savefig(image_root / f"{col}_cuberoot.png")
    plt.close()

In [18]:
df_chaco.median(axis=0)

AV_2        0.041617
VA_4        0.061340
VLa_5       0.079190
VLP_6       0.084642
VPL_7       0.096878
Pul_8       0.104743
LGN_9       0.065870
MGN_10      0.118611
CM_11       0.090956
MD_Pf_12    0.056263
dtype: float64

In [4]:
data = df[df['dz_type2'] == "MS"]
model_data = utils.zscore(data)
for struct in hips_thomas_ref[thalamic_nuclei]:
    outcome = f"{struct}_chaco_curoot"
    formula = f"{outcome} ~ T2LV_curoot + age + Female + tiv"
    res = smf.ols(formula, data=model_data).fit()
    print(res.summary())

                            OLS Regression Results                            
Dep. Variable:      AV_2_chaco_curoot   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.798
Method:                 Least Squares   F-statistic:                     451.3
Date:                Tue, 24 Feb 2026   Prob (F-statistic):          2.64e-156
Time:                        17:42:17   Log-Likelihood:                -281.00
No. Observations:                 457   AIC:                             572.0
Df Residuals:                     452   BIC:                             592.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0059      0.021     -0.281      

In [5]:
hips_thomas_ref

index
1     THALAMUS_1
2           AV_2
4           VA_4
5          VLa_5
6          VLP_6
7          VPL_7
8          Pul_8
9          LGN_9
10        MGN_10
11         CM_11
12      MD_Pf_12
13         Hb_13
14        MTT_14
26        Acc_26
27        Cau_27
28        Cla_28
29        GPe_29
30        GPi_30
31        Put_31
32         RN_32
33         GP_33
34        Amy_34
Name: struct, dtype: object